In [14]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

# Load the dataset
data = pd.read_csv('New heart.csv')

In [15]:
# Handle missing values (Replace 'M' with NaN)
data = data.replace('M', np.nan)

# Drop rows with missing target values (assuming 'target' is the column name)
data.dropna(subset=['HeartDisease'], inplace=True)

In [16]:
# Define features (X) and target (y)
X = data.drop('HeartDisease', axis=1)
y = data['HeartDisease']

In [17]:
# Convert non-numeric columns to numeric using LabelEncoder
label_encoder = LabelEncoder()
X['Sex'] = label_encoder.fit_transform(X['Sex'])
X['ChestPainType'] = label_encoder.fit_transform(X['ChestPainType'])
X['RestingECG'] = label_encoder.fit_transform(X['RestingECG'])
X['ExerciseAngina'] = label_encoder.fit_transform(X['ExerciseAngina'])
X['ST_Slope'] = label_encoder.fit_transform(X['ST_Slope'])

In [18]:
# Handle missing values in the feature columns
imputer = SimpleImputer(strategy='mean')
X = imputer.fit_transform(X)

In [19]:
# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [20]:
# Standardize the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [21]:
# Reshape the data for RNN (assuming you want a time step of 1)
X_train = X_train[:, np.newaxis, :]
X_test = X_test[:, np.newaxis, :]

In [22]:
# Build the RNN model
model = Sequential()
model.add(LSTM(units=64, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(units=1, activation='sigmoid'))

In [23]:
# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [24]:
# Train the model
model.fit(X_train, y_train, epochs=50, batch_size=16, validation_split=0.1)

Epoch 1/50
42/42 [==============================] - 3s 16ms/step - loss: 0.6240 - accuracy: 0.7894 - val_loss: 0.5735 - val_accuracy: 0.8243
Epoch 2/50
42/42 [==============================] - 0s 3ms/step - loss: 0.5024 - accuracy: 0.8621 - val_loss: 0.4951 - val_accuracy: 0.8243
Epoch 3/50
42/42 [==============================] - 0s 3ms/step - loss: 0.4231 - accuracy: 0.8561 - val_loss: 0.4538 - val_accuracy: 0.8243
Epoch 4/50
42/42 [==============================] - 0s 3ms/step - loss: 0.3764 - accuracy: 0.8606 - val_loss: 0.4432 - val_accuracy: 0.8243
Epoch 5/50
42/42 [==============================] - 0s 4ms/step - loss: 0.3539 - accuracy: 0.8606 - val_loss: 0.4398 - val_accuracy: 0.8243
Epoch 6/50
42/42 [==============================] - 0s 3ms/step - loss: 0.3396 - accuracy: 0.8652 - val_loss: 0.4435 - val_accuracy: 0.8243
Epoch 7/50
42/42 [==============================] - 0s 3ms/step - loss: 0.3338 - accuracy: 0.8682 - val_loss: 0.4439 - val_accuracy: 0.8243
Epoch 8/50
42/42 [=

In [25]:
# Make predictions on the test set
y_pred = model.predict(X_test)
y_pred = (y_pred > 0.5).astype(int)

6/6 [==============================] - 0s 3ms/step


In [26]:
# Calculate evaluation metrics
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

# Print the evaluation metrics
print("Precision:", precision*100)
print("Recall:", recall*100)
print("Accuracy:", accuracy*100)
print("F1 Score:", f1*100)


Precision: 95.0
Recall: 88.78504672897196
Accuracy: 90.76086956521739
F1 Score: 91.78743961352657
